#Démo pour l'embarquement d'un réseau de neuronnes sur Maixduino

Ce fichier est un fichier colab, qui va nous permettre de combiner entre des texts et des lignes de code. Colab est généralement utilisé pour des projets d'intelligence artificielle, en nous donnant la possibilité de travailler avec un GPU et une mémoire RAM importante, la possibilité de partage.

Colab nous permet aussi d'utiliser plusieurs bibliothèques de python, pour construire des models de Deep learning, des courbes, des analyses, ou des simples transformations de données. La ligne suivante va nous permettre de voir une des fonctionnalités que colab nous permet de faire. 

Le bloc de code suivant est un script Python qui utilise deux bibliothèques, numpy une bibliothèque de manipulation de matrices et tableaux multidimmensionnels,et matplotlib pour générer des courbes ou  afficher des images, ce script va générer une courbe random et la dessiner en utilisant la bibliothèque matplotlib.

Vous n'avez qu'à démarer la ligne suivante pour voir ce fontionnement.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

ys = 200 + np.random.randn(100)
x = [x for x in range(len(ys))]

plt.plot(x, ys, '-')
plt.fill_between(x, ys, 195, where=(ys > 195), facecolor='g', alpha=0.6)

plt.title("Sample Visualization")
plt.show()

### Commandes shell

Parmi les fonctionnalités que colab nous offre aussi, la possibilité de manipuler l'environnement d'exécution, comme la création d'un fichier ou un dossier en utilisant des commandes shell, ou installer des nouvelles librairies.
Les commandes shell doivent être précédées d'un point d'exclamation.

In [ ]:
!pwd

In [ ]:
!mkdir embedded
!touch /content/embedded/main.c

Vous pouvez vérifier la création de ce dossier, ce dossier contient un fichier main.c

#Machine learning

le Machine-learning ou l'apprentissage automatique sont des techniques qui permettent à la machine d'apprendre à faire des prédiction avec l'entrainnement. le codage normal, est de donner à la machine des régles claires à respecter et lui donner après une entrée pour vous donner la sortie, mais le machine-learning, qui fait partie de l'intelligence artificielle, utilise une autre technique, c'est de donner à la machine des entrées et leurs sorties et laisser la machine deviner les régles pour obtenir la même sortie, au fur et à mesure la machine commence à donner des prédictions très précises.

Le deep-learning ou l'apprentissage profond est une des branches du machine-learning qui nous permet de créer des réseaux de neuronnes afin de prédire des réponses complexes.

Nous allons créer un réseau de neuronnes simple pour prédire la fonction f(x)=2x+0.1, pour cela nous allons pas décrir la fonction, mais nous allons donner au réseau des exemples d'entrées avec leurs sorties.

Les réseaux de neuronnes donnent des meilleurs résultats, si toutes les variables sont entre 1 et -1, alors je vais travailler avec des -1<x<1.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

X = np.array([0,0.1,0.2,-0.4,-0.7,0.134,0.353,0.356, 0.-145,-0.4])
Y = np.empty(shape=0, dtype=float)

# Calculate Y-Values
for x in X:
    Y = np.append(Y, float(2*x +0.1))


# model architecture
model = keras.Sequential()
model.add(keras.layers.Dense(1, input_shape=(1,)))
model.add(keras.layers.Dense(8))
model.add(keras.layers.Activation("linear"))
model.add(keras.layers.Dense(4))
model.add(keras.layers.Activation("linear"))
model.add(keras.layers.Dense(1))
model.add(keras.layers.Activation("linear"))
# compile model
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['accuracy'])

# train model
model.fit(X, Y, epochs=250, batch_size=10)

Regardez les predictions suivantes et les comparer avec les résultats corrects.

In [ ]:
y_pred = model.predict([0,0.1, 0.3, 0.34, 0.12])
print(y_pred)

#Préparation de la base de données

Avant de commencer, soyez sûr que vous utilisez un GPU


Exécution -> Modifier le type d'exécution -> GPU -> enregistrer

Maintenant, après une petite introduction de ce que nous désignons par le machine learning, nous allons entrer dans le vif du sujet, d'ailleurs le sujet est l'embarquement d'un réseau de neuronnes sur la carte MaixDuino, nous commençons par la préparation de la data, sans data le machine learning fonctionnera jamais.

Pour cela, nous allons utiliser une base de donnée qui s'appelle CelebA, et qui contient 202k images, vous pouvez la trouver sur le site web http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html , vous pouvez télécharger la base de données, et la charger sur colab. Pour notre cas, nous allons utiliser un quart de cette base, car travailler avec toute la base est impossible, vu les quotas imposés par Google.

La base de données est déjà prête, vous n'avez qu'à récupérer deux fichier .zip et les décompresser, chaque dossier contient deux dossier "train" et "val", train contient 19999 images ou 19999 fichiers xml, et val contient 2000 images ou 2000 fichiers xml, chaque fichier xml contient les coordonnées des cadres des visages, vous pouvez ouvrir un fichier xml, et analyser les informations qu'il contient.

Téléchargez les deux bases de données qui seront fournies, et décompressez ces bases avec les deux lignes de code suivantes:

In [ ]:
!unzip /content/celebaForModel-20210111T145135Z-001.zip
!unzip /content/celebaForModelXML-20210111T150438Z-001.zip

Vous pouvez remarquer que y a deux dossier qui viennent de s'ajouter.


#aXeleRate

Axelerate est un projet open source fait par un ingénieur russe qui s'appelle Dmitry MASLOV, qui travaille dans le domaine de l'intelligence artificielle embarquée, M.MASLOV a participé à la réalisation de ce projet.

La carte Maixduino n'accepte que des fichier kmodels, car le kpu a une configuration particulière, pour créer un model kmodel nous devons créer un modèle h5 avec Tensorflow, après nous le transformons en fichier tflite, qui utilise des int de 16-bits à la place de 32-bits ce qui réduit la taille du model par 2, au moins. Ensuite le fichier tflite sera transformé en fichier kmodel. Tout cela sera fait par aXeleRate.

Axelerate est un outil pour créer des fichier kmodel, elle fait toutes les transformations nécessaires, et il suffit de préciser ce que nous voulons faire à l'aide d'un fichier de configuration ou un dictionnaire de configuration.

Nous commonçons par ajouter la bibiliothèque aXeleRate, nous ajoutons aussi une ancienne version de la bibliothèque imgaug, pour éviter des conflits de versions.

imgaug est une bibiothèque pour l'augmentation de data, si par exemple vous avez une image d'une fleur vous pouvez avoir 5 images de cette fleur en appliquant une transformation miroir, tourner l'image de 20° ainsi de suite.

In [ ]:
!pip uninstall -y imgaug && pip uninstall -y albumentations && pip install imgaug==0.4
!git clone https://github.com/AIWintermuteAI/aXeleRate.git
import sys
sys.path.append('/content/aXeleRate')
from axelerate import setup_training, setup_inference

#Visualisation des images
Nous pouvons utiliser la fonction visualize_detection_dataset pour visualiser notre base de données, et visualiser quelques exemple de l'augmentation de data

In [ ]:
from axelerate.networks.common_utils.augment import visualize_detection_dataset

visualize_detection_dataset(img_folder='/content/celebaForModel/train', ann_folder='/content/celebaForModelXML/train', num_imgs=10, img_size=320, augment=True)

#File configuration

Pour notre cas, c'est plus simple d'utiliser un dictionnaire à la place d'un fichier de configuration d'aXeleRate, mais le contenu est le même, dans ce fichier de configuration nous allons préciser à aXeleRate comment elle doit construire le model, comment elle doit l'entrainer et où elle doit le stocker ou l'enregistrer.

Pour bien comprendre ce qui va se passer après, nous expliquons les keys du dictionnaire:

- type c'est le type de model que nous voulons faire, Detector, Segnet ou Classifier, pour notre cas c'est un detecteur de visage alors nous allons choisir Detector.
-architecture, c'est l'architecture du model, y'en a plusieurs architecture que les experts ont démontré leur efficacité. Les architectures que aXeleRate soutienne sont: Full Yolo - Tiny Yolo - MobileNet1_0 - MobileNet7_5 - MobileNet5_0 - MobileNet2_5 - SqueezeNet - VGG16 - ResNet50, avec notre carte nous ne pouvons pas utiliser des architectures très développées, notre model ne doit pas dépassé la taille 2MB, alors nous devons choisir entre Tiny Yolo, MobileNet(jusqu'à alpha 0.75) et SqueezeNet.
-input_size, c'est la taille des entrées, y a deux possibilités 224x224(224) ou 240x320(320), avec les models classiques, l'entrainement avec des images carrées est plus efficace que l'entrainement avec des images qui ne sont pas carrées.
-anchors, c'est une spécialité de Yolo, pour faire la différence entre les différents cadres(https://towardsdatascience.com/anchor-boxes-the-key-to-quality-object-detection-ddf9d612d4f9)
-labels, c'est les labels que notre base de données contient, vous pouvez lire un des fichier xml, que la base de données contient, pour mieux comprendre.
-les 4 scales, sont pour fixer à quel point le model doit pénaliser des fausses prédictions.
-weights:

  * full: si nous avons un même model pré-entrainé, nous pouvons utiliser ce model pour l'entrainer de plus
  *backend: des weights pré-entrainés sur une autre tâche, mais ils sont simple plus à réentrainer, cela s'appelle le transfert de l'apprentissage.
-actual-epoch, combien d'epochs nous aimerions faire entrainer notre model
-train_image_folder, le dossier contenant les images d'entrainement
-train_annot_folder, le dossier contenant les réponses, les positions de cadres
-train_times, combien de fois nous voulons répéter l'entrainement
-val_image_folder, le dossier contenant les images de validation
-val_annot_folder, le dossier contenant les réponses, les positions de cadres
-valid_times, combien de fois nous voulons répéter la validation
-batch_size, la taille des batchs d'entrainnement
-learning_rate, le coefficient d'entrainement initial
-saved_folder, l'endroit où nous voulons enregistrer le model
-first_trainable_layer, le premier layer à entrainer(au cas où nous voulons pas entrainer les premiers layers du modèle)
-augmentation, l'augmentation de la base des données

La majorité de ces données ne sont pas à changer, les seules informations que nous pouvions changer sont les endroits où nous avons la bases de données, l'endroit où nous voulons enregistrer le model résultat.

In [ ]:
!mkdir kendryteModel

config = {
        "model":{
            "type":                 "Detector",
            "architecture":         "MobileNet7_5",
            "input_size":           224,
            "anchors":              [0.57273, 0.677385, 1.87446, 2.06253, 3.33843, 5.47434, 7.88282, 3.52778, 9.77052, 9.16828],
            "labels":               ["face"],
            "coord_scale" : 		1.0,
            "class_scale" : 		1.0,
            "object_scale" : 		5.0,
            "no_object_scale" : 	1.0
        },
        "weights" : {
            "full":   				"",
            "backend":   		    "imagenet"
        },
        "train" : {
            "actual_epoch":         15,
            "train_image_folder":   "/content/celebaForModel/train",
            "train_annot_folder":   "/content/celebaForModelXML/train",
            "train_times":          1,
            "valid_image_folder":   "/content/celebaForModel/val",
            "valid_annot_folder":   "/content/celebaForModelXML/val",
            "valid_times":          1,
            "valid_metric":         "mAP",
            "batch_size":           32,
            "learning_rate":        1e-4,
            "saved_folder":   		F"/content/kendryteModel",
            "first_trainable_layer": "",
            "augumentation":				True,
            "is_only_detect" : 		False
        },
        "converter" : {
            "type":   				["k210","tflite"]
        }
    }

Tensorboard pour effectuer des statistics



In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs
!sleep 10

Vérifier que vous utiliser un GPU, et quel GPU vous utiliser

Un entrainement sans GPU est extrêmement long

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

Maintenant nous allons commencer l'entrainement de notre model, en utilisant axelerate, cette opération va probabelement prendre énormément du temps, et si vous n'avez pas précisé l'utilisation d'un GPU, vous devez le faire.

Lancer le code suivant et regardez les premiers messages pour comprendre encore mieux, quand l'entrainement commence, prenez un café d'une heure 20 min, et vérifier de temps en temps que le model apprend tout le temps, car nous risquons de dépassé les quotas de Colab en terme de nombre d'utilisation du matériel.

In [ ]:
from keras import backend as K 
K.clear_session()
model_path = setup_training(config_dict=config)

Après la fin de l'entrainement, vérifier bien les performances, et dessiner des courbes de performances si vous voulez, afin de vérifier que notre model fonctionne correctement, nous allons faire des inférences sur les données de la validation.

NB: mAP=moyen Average Precision

NB: Le modèle ne s'entraine pas sur les images de validation, mais il les utilise pour vérifier s'il apprend bien à prédire des réponses qui n'a jamais vu.

alors, démarez la ligne suivante, et regardez si votre model fonctionne bien.

Pensez à stopper la ligne après quelques images, car cela ne va jamais arrêter avec 2000 images.

In [ ]:
%matplotlib inline
from keras import backend as K 
K.clear_session()
setup_inference(config, model_path)

#Kflash_gui et maixpyide

Maintenant, nous avons le fichier kmodel, vous devez ouvrir kendryteModel et ouvrir le dernier dossier, normalement y a qu'un dossier mais si vous avez effectué deux fois l'entrainement, vous devez ourir le dernier dossier; télécharger le fichier kmodel, pour le brûler sur la carte.

Télécharger la biblithèque Kflash_gui pour brûler le fichier kmodel sur la carte https://github.com/sipeed/kflash_gui avec git clone, vérifier si le dossier kflash_py n'est pas vide, si c'est le cas, télécharger la bibliothèque kflash_py à partir de ce lien: https://github.com/sipeed/kflash.py/tree/7307f75c962f15392a54620715e22ab5da97b614 .

Maintenant, démarez le script avec python3 kflash_gui.py, si cela ne marche pas probablement vous aurez besoin de télécharger quelques bibliothèques Python (avec pip3), comme PyQt5.

Quand ce marche bien vous aurez une fenêtre, vous devez préciser quel fichier vous voulez brûler, et c'est clairement le fichier kmodel téléchargé précédemment, au quel endroit vous voulez brûler le script, choisissez 0x300000, précisez que vous travaillez avec Maixduino et à quel port la carte est connectée, généralement ttyUSB0, cliquez ensuite sur Download.

quand vous aurez fait cela, téléchargez maixpyIDE, qui est un IDE de sipeed, suivez le tutoriel suivant https://maixpy.sipeed.com/en/get_started/maixpyide.html , démarer le programme avec ./bin/maixpyide, connectez vous à la carte, et lancez le script suivant sur l'IDE(pas sur colab).


NB: si vous n'arrivez pas à accéder au port ttyUSB0, penser à changer les permissions, sudo chmod 777 /dev/ttyUSB0

In [ ]:
import sensor
import image
import lcd
import KPU as kpu

lcd.init()
sensor.reset()

sensor.set_pixformat(sensor.RGB565)
sensor.set_framesize(sensor.QVGA)
sensor.set_windowing((224, 224))
#sensor.set_vflip(1)
sensor.run(1)
classes = ["face"]
task = kpu.load(0x300000)
a = kpu.set_outputs(task, 0, 7,7,30)
anchor = (1.889, 2.5245, 2.9465, 3.94056, 3.99987, 5.3658, 5.155437, 6.92275, 6.718375, 9.01025)
a = kpu.init_yolo2(task, 0.3, 0.3, 5, anchor)
while(True):
    img = sensor.snapshot()
    code = kpu.run_yolo2(task, img)
    if code:
        for i in code:
            a = img.draw_rectangle(i.rect())
    a = lcd.display(img)
    print(code)
a = kpu.deinit(task)